In [1]:
from tensorflow import keras

print(keras.layers.Conv2D)

<class 'keras.src.layers.convolutional.conv2d.Conv2D'>


In [1]:
import numpy as np
import cv2

print()
# Initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Open input video file
input_file = r'Videos\vid.mp4'
cap = cv2.VideoCapture(input_file)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output video file
out = cv2.VideoWriter(
    'output.avi',
    cv2.VideoWriter_fourcc(*'MJPG'),
    fps,
    (frame_width, frame_height)
)

while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        break

    # Resizing for faster detection
    frame = cv2.resize(frame, (640, 480))
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Adjust HOG parameters for better detection
    hogParams = {
        'winStride': (8, 8),   # Default is (8,8)
        'padding': (16, 16),   # Default is (8,8)
        'scale': 1.05,         # Default is 1.05
        'hitThreshold': 0.45   # Default is 0 (adjustable, higher values reduce false positives)
    }

    # Detect people in the image
    # Returns the bounding boxes for the detected objects
    boxes, weights = hog.detectMultiScale(frame, **hogParams)

    boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])

    for (xA, yA, xB, yB) in boxes:
        # Display the detected boxes in the colour picture
        cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)

    # Write the output video
    out.write(frame)

    # Display the resulting frame
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


-1

### Mobile Net Model


In [2]:
import cv2
import numpy as np
import tensorflow as tf
import pathlib  # Import pathlib module for handling paths
import os      # Import os module for file operations

# Function to load a TensorFlow model
def load_model(model_name):
    # Get the current directory
    current_dir = pathlib.Path().resolve()

    # Define the base URL and model file name
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'

    # Get the model directory path where it will be saved or has been saved
    model_dir = tf.keras.utils.get_file(
        fname=model_name, 
        origin=base_url + model_file,
        untar=True,
        cache_subdir=current_dir)  # Save in the current directory

    # Set the model directory path to include the saved_model subdirectory
    model_dir = pathlib.Path(model_dir) / "saved_model"
    return model_dir

# Function to perform object detection
def detect_objects(frame, detection_function):
    input_tensor = tf.convert_to_tensor(frame)
    input_tensor = input_tensor[tf.newaxis,...]

    detections = detection_function(input_tensor)

    return detections

# Function to draw bounding boxes and labels
def draw_boxes(frame, detections, confidence_threshold=0.5):
    height, width, _ = frame.shape

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    for i in range(num_detections):
        score = detections['detection_scores'][i]
        class_id = int(detections['detection_classes'][i])
        bbox = detections['detection_boxes'][i]

        if score < confidence_threshold or class_id != 1:
            continue

        ymin, xmin, ymax, xmax = bbox
        xmin = int(xmin * width)
        xmax = int(xmax * width)
        ymin = int(ymin * height)
        ymax = int(ymax * height)

        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.putText(frame, 'Person', (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    return frame

# Main function for object detection in video
def main():
    # Load SSD-MobileNet model
    model_name = 'ssd_mobilenet_v2_coco_2018_03_29'
    model_dir = load_model(model_name)

    # Check if the model directory exists
    if not os.path.exists(model_dir):
        raise FileNotFoundError(f"Model directory '{model_dir}' not found. Make sure the model has been downloaded correctly.")

    # Load the detection function from the saved model
    detection_model = tf.saved_model.load(str(model_dir))
    detection_function = detection_model.signatures['serving_default']

    # Load video
    cap = cv2.VideoCapture(r"Videos\dogvid.mp4")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.resize(frame, (640, 480))  # Resize frame as needed

        # Perform object detection
        detections = detect_objects(frame, detection_function)

        # Draw bounding boxes and labels on the frame (only for persons)
        frame_with_boxes = draw_boxes(frame.copy(), detections)

        # Display the frame with detected person
        cv2.imshow('Person Detection', frame_with_boxes)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
